In [1]:
import matplotlib.pyplot as plt
import numpy as np

from pdf import PdfConverter
import textract

import glob
import nltk
import gensim

#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download() # for all

from nltk.stem import PorterStemmer, WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import numpy as np

In [ ]:
data = []

excludes_pre = "introduction abstract".split()
excludes_post = "acknowledge references bibliography".split()

regtoken = RegexpTokenizer(r'[a-z]\w{3,}') # at least of length four, starting with letter (no pure number)

stop_words = set(stopwords.words('english'))
stop_words.update({'et', 'al', 'fig', 'ref', 'tab'})

for path in sorted(glob.glob('my_pub/*.pdf')):
  try:
    pdf = PdfConverter(file_path=path)
    doc = pdf.extractTxt()
  except:
    doc = textract.process(path, method='pdfminer').decode()

  doc = doc.lower()
  doc = re.sub('\s{0,}-\s{0,}', '', doc)  # hyphen
    
  idx = 0
  for ex in excludes_pre:
    try:
       idx = max(doc.index(ex), idx)
    except:
      pass
  doc = doc[idx:]
  
  idx = len(doc)
  for ex in excludes_post:
    try:
       idx = min(doc.index(ex), idx)
    except:
      pass

  words = regtoken.tokenize(doc[:idx])   
  words = [w for w in words if not w in stop_words ]
  data.append(words)

In [ ]:
import pickle
pickle.dump(data, open('data_0.pkl', 'wb'))
            
porter = PorterStemmer()
data = [ [porter.stem(w) for w in doc] for doc in data ]

In [ ]:
from gensim import models, corpora

NUM_TOPICS = 7

# Build a Dictionary - association word to numeric id
dictionary = corpora.Dictionary(data)
 
# Transform the collection of texts to a numerical form
corpus = [dictionary.doc2bow(doc) for doc in data]
 
# Have a look at how the first document looks like: [(word_id, count), ...]
#print(corpus[0])
#[(0, 3), (1, 1), (2, 1), (3, 1), (4, 3), (5, 1), (6, 6), (7, 1), ... 

# Build the LDA model
lda_model = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)

# Build the LSI model
lsi_model = models.LsiModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)

print("LDA Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lda_model.print_topic(idx, 10))
 
print("=" * 20)
 
print("LSI Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lsi_model.print_topic(idx, 10))
 
print("=" * 20)